In [1]:
import pandas as pd
import numpy as np
import cv2
import sys
import caffe
import pickle
from caffe.proto import caffe_pb2
import random

In [2]:
#Naive Noisy Binary Search
def NNBS(S,e, epsilon, accuracies, P, longitud1):
    global longitud
    global P1
    P1=P
    longitud=longitud1
    size=len(S)
    thresh=0.548028
    
    ll=1
    uu=size
    while ((uu-ll)>0):
        mm=int(np.ceil((ll+uu)/2.0))
        
        if S[mm-1]==16:
                hp_wins=0
        elif S[mm-1]==40:
                hp_wins=1
        else:
            ind2=np.random.permutation(longitud[S[mm-1]])
            ind2=ind2[0:accuracies[S[mm-1]]]
            
            img2=P1[S[mm-1]][ind2]
            hp_wins=dataset_and_compare(img2, e, thresh)

        if (0.5-epsilon)<=hp_wins and hp_wins<=0.5+epsilon:
            x=mm-1
            break
        else:
            if hp_wins<(0.5-epsilon):
                if mm==1:
                    x=0
                    break
                ll=np.ceil((ll+uu)/2.0)
            else:
                if (mm==size):
                    x=size-1
                    break
                uu=np.ceil((ll+uu)/float(2))
            if (uu-ll==1) & (mm==uu):
                x=int(ll)-1
                break
    if (uu-ll)==0:
        x=int(ll)-1
    return S[int(x)]


def dataset_and_compare(img2,ej, thresh):
    

    img2=img2.values
    images=np.zeros((len(img2), 6, 224, 224))
    dim=len(img2)
    index=0
    for ina in img2:
        img_path_1='./Anchors/Images/'+ina+'.jpg'  
        img1 = cv2.imread(img_path_1, cv2.IMREAD_COLOR)
        if img1.shape!=(224,224,3):
            img1 = cv2.resize(img1,(224,224), interpolation = cv2.INTER_CUBIC)        
        if img1==None:
            dim=dim-1
            images = images[0:dim-1, :, :]
            continue
        
        img=np.concatenate((img1, ej), axis=2)
        img = img.transpose((2,0,1))
        images[index,:,:,:]=img
        index=index+1
    out = net0.forward_all(pair_data = images )
    bla=sum(out['prob'][:,1]>thresh)/float(len(out['prob']))
 

    return bla

In [3]:
with open('./Anchors/table_anchors.pkl', 'rb') as f:
    [df, longitud]=pickle.load(f)


epsilon=0.03
budget=50
GPU_ID=0

caffe_root = './caffe/'  
sys.path.insert(0, caffe_root + 'python')
caffe.set_mode_gpu()
caffe.set_device(GPU_ID)
model_def = './Deployment_prototxt_files/AgeNet-based_Mode_I.prototxt'
model_weights0 = './Models/AgeNet-based_Mode_I.caffemodel'


net0 = caffe.Net(model_def,      # defines the structure of the model
                model_weights0,  # contains the trained weights
                caffe.TEST)     


roc_values=np.array([0.691794793548, 0.725495077349, 0.729365110244, 0.775528180484])
roc_values=(1-roc_values)/sum(1-roc_values)

accuracies={}
accuracies[18.5]=roc_values[0]
accuracies[25]=roc_values[1]
accuracies[30]=roc_values[2]
accuracies[35]=roc_values[3]
for key in accuracies:    
    accuracies[key] =  int(accuracies[key]*budget)
S=[16, 18.5, 25, 30, 35, 40]


df4_neutral = pd.DataFrame(columns=('Athlete', 'Ground truth (BMI)', 'Prediction'))
Names=['LIM HYOJUN','DAVID GLEIRSCHER']
BMI=[22.67, 26.87]

for tra in range(2):
            img_path_1='./Testing_images/'+str(tra)+'.jpg'
            img1= cv2.imread(img_path_1, cv2.IMREAD_COLOR)
            pred=NNBS(S,img1, epsilon, accuracies, df, longitud)
            df4_neutral.loc[tra]=[Names[tra], BMI[tra], pred]

df4_neutral.loc[(df4_neutral['Prediction'] ==16),'Prediction'] = 'Underweight (16-18.5)'
df4_neutral.loc[(df4_neutral['Prediction'] ==18.5),'Prediction'] = 'Normal (18.5-25)'
df4_neutral.loc[(df4_neutral['Prediction'] ==25),'Prediction'] = 'Overweight (25-30)'
df4_neutral.loc[(df4_neutral['Prediction'] ==30),'Prediction'] = 'Moderately obese (30-35)'
df4_neutral.loc[(df4_neutral['Prediction'] ==35),'Prediction'] = 'Severely obese (35-40)'

/home/lpolania/enter/lib/python2.7/site-packages/ipykernel/__main__.py:60: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


In [4]:
#Results
df4_neutral

,Athlete,Ground truth (BMI),Prediction
0,LIM HYOJUN,22.67,Normal (18.5-25)
1,DAVID GLEIRSCHER,26.87,Overweight (25-30)
